In [18]:
!pip install pyDOE

#you need to add a dataset with GaussJacobiQuadRule_V3 on the right 
import sys
# sys.path.insert(1, '/kaggle/input/quadrule')
from GaussJacobiQuadRule_V3 import Jacobi, DJacobi, GaussLobattoJacobiWeights, GaussJacobiWeights

#import GaussJacobiQuadRule_V3


import tensorflow as tf
print(tf.version)
#tf.disable_v2_behavior()
import pyDOE

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from pyDOE import lhs
#from GaussJacobiQuadRule_V3.py import Jacobi, DJacobi, GaussLobattoJacobiWeights, GaussJacobiWeights
import time

<module 'tensorflow._api.v2.version' from '/home/mariano/Documenti/test/base/lib/python3.9/site-packages/tensorflow/_api/v2/version/__init__.py'>


In [19]:
Net_layer = [1] + [20] * 4 + [1]
print(Net_layer)

[1, 20, 20, 20, 20, 1]


In [20]:
class VPINN:
    def __init__(self,Net_layer):

        self.layers=Net_layer


        self.weights,self.biases=self.initialize_NN(self.layers)

        self.variables=[self.weights,self.biases]


    def initialize_NN(self,layers):        
        weights = []
        biases = []
        self.layers=layers
        num_layers = len(layers) 
        #transpose everything to have the classic form y=W*x+b 
        for l in range(0,num_layers-1):

            in_dim=layers[l]
            out_dim=layers[l+1]

            xavier_stddev=np.sqrt(2/(in_dim + out_dim))

            W = tf.Variable(tf.random.truncated_normal([out_dim,in_dim], stddev=xavier_stddev,dtype=tf.float64), dtype=tf.float64,trainable=True)
            b = tf.Variable(tf.zeros([layers[l+1],1], dtype=tf.float64), dtype=tf.float64,trainable=True)
            weights.append(W)
            biases.append(b)        
        return weights, biases
    


    @tf.function
    def neural_net(self,X): #marked for future develop
        weights=self.weights
        biases=self.biases 
        num_layers = len(weights) + 1
        H = X 
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            H = tf.sin((W @ H)+b) #change here for having different activation function
        W = weights[-1]
        b = biases[-1]
        Y = (W @ H)+b
        return Y
    

    
    @tf.function
    def net_du(self,X):
        with tf.GradientTape() as tape:
            tape.watch(X)
            with tf.GradientTape() as tape_:
                tape_.watch(X)
                y=self.neural_net(X)
            d1=tape_.gradient(y,X)
        d2=tape.gradient(d1,X)
        return d1,d2
    

    def predict(self, x):
        x=np.array(x)
        x=tf.constant(x[:, np.newaxis], dtype=tf.float64)
        u_pred  = self.net_du(x)
        return u_pred  
    

    def Test_fcn(self, N_test,x):
        test_total = []
        for n in range(1,N_test+1):  
            test  = Jacobi(n+1,0,0,x) - Jacobi(n-1,0,0,x)
            test_total.append(test)
        return np.asarray(test_total)

    def dTest_fcn(self, N_test,x):  #valuete the first and second derivatives of test functions on a point x 
        d1test_total = []
        d2test_total = []
        for n in range(1,N_test+1):  
            if n==1:
                d1test = ((n+2)/2)*Jacobi(n,1,1,x)
                d2test = ((n+2)*(n+3)/(2*2))*Jacobi(n-1,2,2,x)
                d1test_total.append(d1test)
                d2test_total.append(d2test)
            elif n==2:
                d1test = ((n+2)/2)*Jacobi(n,1,1,x) - ((n)/2)*Jacobi(n-2,1,1,x)
                d2test = ((n+2)*(n+3)/(2*2))*Jacobi(n-1,2,2,x)
                d1test_total.append(d1test)
                d2test_total.append(d2test)    
            else:
                d1test = ((n+2)/2)*Jacobi(n,1,1,x) - ((n)/2)*Jacobi(n-2,1,1,x)
                d2test = ((n+2)*(n+3)/(2*2))*Jacobi(n-1,2,2,x) - ((n)*(n+1)/(2*2))*Jacobi(n-3,2,2,x)
                d1test_total.append(d1test)
                d2test_total.append(d2test)    
        return np.asarray(d1test_total), np.asarray(d2test_total)
    
    def Test_f(self,n,x):
        test  = Jacobi(n+1,0,0,x) - Jacobi(n-1,0,0,x)
        return test

    def print_net(self):
        for e in range(0,len(self.layers)-1):
            print("layer ",e, " :")
            print(self.weights[e])
            print(self.biases[e])
    
    def train(self,nIter,total_record):
     #maybe like variables sess.run will do the rest, u(boundary cond) is assigned to u_tf ,x(boundary) is assigned to x_tf and so on
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        lossb_weight=1.0
        start_time       = time.time()

        for epoch in range(nIter+1):
                with tf.GradientTape() as tape:
                    tape.watch(self.weights[0])
                    loss_valueb,loss_valuev=self.calculate_loss()
                loss=loss_valueb*lossb_weight+loss_valuev
                
                gradients = tape.gradient(loss,self.weights[0])
                print(gradients)
                optimizer.apply_gradients(zip(gradients,self.weights[0]))
            
                # Print training loss for this epoch
                if (epoch) % 100 == 0:
                            elapsed = time.time() - start_time
                            str_print = 'It: %d, Lossb: %.3e, Lossv: %.3e, Time: %.2f'
                            print(str_print % (epoch, loss_valueb, loss_valuev, elapsed))
                            start_time = time.time()        
                            total_record.append(np.array([epoch, loss,loss_valueb,loss_valuev]))
        return total_record
    
    def calculate_loss(self):
        varloss_total = 0.0
        for e in range(N_Element):
            F_ext_element  = F_ext_total[e]
            Ntest_element  =tf.shape(F_ext_element)[0] #for each element of the grid you have a vector of the focing term (suppose its n-loc),so the you can have at most quad formula n_loc
            
            x_quad_element = grid[e] + (grid[e+1]-grid[e])/2*(x_quad+1)
            #to change change integral to the ref segment in (-1,1)
            jacobian= (grid[e+1]-grid[e])/2

            x_b_element    = np.array([[grid[e]], [grid[e+1]]])


            test_quad_element =tf.constant(self.Test_fcn(Ntest_element, x_quad),dtype=tf.float64)

            d1test_quad_element, d2test_quad_element = tf.constant(self.dTest_fcn(Ntest_element,x_quad),dtype=tf.float64)

            d1test_bound_element, d2test_bounda_element = tf.constant(self.dTest_fcn(Ntest_element, np.array([[-1],[1]])),dtype=tf.float64)






            U_NN_element = tf.reshape(tf.stack([tf.reduce_sum(w_quad*d1u_NN_quad_element*d1test_quad_element[i]) \
                                                for i in range(Ntest_element)]),(-1,1)) 

                


            Res_NN_element = U_NN_element - F_ext_element
            loss_element = tf.reduce_mean(tf.square(Res_NN_element))
            varloss_total = varloss_total + loss_element





        lossb = tf.reduce_mean(tf.square(self.neural_net(X_bound) - u_bound))   #u_NN_pred is what your network has calc,while u_tf is the real value
        return lossb,varloss_total

In [21]:
N_Quad=100
N_testfcn=20
N_Element=5
a=-1.0
b=1.0

omega = 8*np.pi
amp = 1.0
r1 = 80.0

def u_exact(x):
    utemp = 0.1*np.sin(omega*x) + np.tanh(r1*x)
    return amp*utemp

def f_ext(x):
    gtemp =  -0.1*(omega**2)*np.sin(omega*x) - (2*r1**2)*(np.tanh(r1*x))/((np.cosh(r1*x))**2)
    return -amp*gtemp




[x_quad,w_quad] = GaussLobattoJacobiWeights(N_Quad, 0, 0)

x_quad=tf.constant([x_quad],dtype=tf.float64)
        

test_quad_element = model.Test_fcn(N_testfcn,x_quad)
d1test_quad_element, d2test_quad_element = model.dTest_fcn(N_testfcn,x_quad)


[x_l, x_r] = [a, b]     #modify in the future if you want general (a,b) interval
delta_x = (x_r - x_l)/N_Element
grid = np.asarray([ x_l + i*delta_x for i in range(N_Element+1)])

        #build right side (it doesnt depend on the network)

F_ext_total = []
for e in range(N_Element):
    x_quad_element = grid[e] + (grid[e+1]-grid[e])/2*(x_quad+1)  #traslation of the element 

    jacobian = (grid[e+1]-grid[e])/2
    testfcn_element = np.asarray([ model.Test_f(n,x_quad)  for n in range(1,N_testfcn+1)])

    #this is fh calculated on the real nodal values summed over each real nodal values(which comes form CGL nodes )

    f_quad_element = f_ext(x_quad_element)
    F_ext_element  = jacobian*np.asarray([sum(w_quad*f_quad_element*testfcn_element[i]) for i in range(N_testfcn)])    
    F_ext_element = F_ext_element[:,None] 
    F_ext_total.append(F_ext_element)

F_ext_total = np.asarray(F_ext_total)
X_bound = np.asarray([a,b],dtype=np.float64)[:,None]
u_bound   =u_exact(X_bound)


X_bound = tf.constant(np.transpose(X_bound),dtype=tf.float64)
u_bound   =tf.constant(np.transpose(u_bound),dtype=tf.float64)

F_ext_total=tf.constant(F_ext_total,dtype=tf.float64)

In [22]:

model=VPINN(Net_layer)




input=tf.constant([[1.0,2.0]],dtype=tf.float64)
print(input)

c=model.neural_net(input)

print(c)


input=tf.constant([[1.0,2.0]],dtype=tf.float64)
print(input)

c=model.neural_net(x_quad)

print(c)



tf.Tensor([[1. 2.]], shape=(1, 2), dtype=float64)
tf.Tensor([[0.44571702 0.70008579]], shape=(1, 2), dtype=float64)
tf.Tensor([[1. 2.]], shape=(1, 2), dtype=float64)
tf.Tensor(
[[-0.44571702 -0.44544999 -0.44482163 -0.44383345 -0.44248472 -0.44077441
  -0.4387012  -0.43626352 -0.4334596  -0.43028746 -0.42674496 -0.42282983
  -0.4185397  -0.41387214 -0.4088247  -0.40339499 -0.3975807  -0.39137965
  -0.38478988 -0.37780971 -0.37043776 -0.36267308 -0.35451519 -0.34596415
  -0.33702064 -0.32768606 -0.31796253 -0.30785307 -0.29736158 -0.28649294
  -0.2752531  -0.2636491  -0.25168913 -0.23938259 -0.22674013 -0.21377364
  -0.20049632 -0.18692261 -0.17306828 -0.15895031 -0.14458693 -0.12999752
  -0.11520259 -0.10022368 -0.08508328 -0.06980472 -0.05441208 -0.03893005
  -0.02338381 -0.00779891  0.00779891  0.02338381  0.03893005  0.05441208
   0.06980472  0.08508328  0.10022368  0.11520259  0.12999752  0.14458693
   0.15895031  0.17306828  0.18692261  0.20049632  0.21377364  0.22674013
   0.2393

In [23]:
print(test_quad_element.dtype)
print(d1test_quad_element.dtype,d2test_quad_element.dtype)

float64
float64 float64


In [24]:
print(grid.dtype)

float64


In [25]:
print(F_ext_total[e])
print(tf.shape(F_ext_total[e])[0])
print(F_ext_total.dtype)
print(X_bound)
print(u_bound)


tf.Tensor(
[[[ 0.00000000e+00 -2.04509619e-05 -1.21754484e-04 ...  2.63252612e-06
    1.30336442e-07  0.00000000e+00]]

 [[ 0.00000000e+00  3.40596651e-05  2.02419924e-04 ...  4.37664158e-06
    2.17066346e-07  0.00000000e+00]]

 [[ 0.00000000e+00 -4.76304942e-05 -2.82331232e-04 ...  6.10445152e-06
    3.03554874e-07  0.00000000e+00]]

 ...

 [[ 0.00000000e+00  2.36659514e-04  1.20581303e-03 ...  2.60716009e-05
    1.50825958e-06  0.00000000e+00]]

 [[ 0.00000000e+00 -2.47660589e-04 -1.23886000e-03 ...  2.67861290e-05
    1.57837075e-06  0.00000000e+00]]

 [[ 0.00000000e+00  2.58388467e-04  1.26742713e-03 ...  2.74037959e-05
    1.64674081e-06  0.00000000e+00]]], shape=(20, 1, 100), dtype=float64)
tf.Tensor(20, shape=(), dtype=int32)
<dtype: 'float64'>
tf.Tensor([[-1.  1.]], shape=(1, 2), dtype=float64)
tf.Tensor([[-1.  1.]], shape=(1, 2), dtype=float64)


In [26]:
x_quad_element = grid[e] + (grid[e+1]-grid[e])/2*(x_quad+1)

print(x_quad_element)



tf.Tensor(
[[0.6        0.60014828 0.60049695 0.60104454 0.60179046 0.60273397
  0.60387411 0.60520976 0.60673959 0.60846205 0.61037545 0.61247787
  0.61476722 0.6172412  0.61989737 0.62273307 0.62574547 0.62893157
  0.6322882  0.63581201 0.63949948 0.64334694 0.64735056 0.65150634
  0.65581014 0.66025768 0.66484451 0.66956607 0.67441765 0.67939441
  0.68449139 0.68970352 0.69502559 0.7004523  0.70597825 0.71159791
  0.7173057  0.72309592 0.7289628  0.73490049 0.74090307 0.74696456
  0.75307892 0.75924005 0.76544181 0.77167802 0.77794246 0.78422889
  0.79053104 0.79684263 0.80315737 0.80946896 0.81577111 0.82205754
  0.82832198 0.83455819 0.84075995 0.84692108 0.85303544 0.85909693
  0.86509951 0.8710372  0.87690408 0.8826943  0.88840209 0.89402175
  0.8995477  0.90497441 0.91029648 0.91550861 0.92060559 0.92558235
  0.93043393 0.93515549 0.93974232 0.94418986 0.94849366 0.95264944
  0.95665306 0.96050052 0.96418799 0.9677118  0.97106843 0.97425453
  0.97726693 0.98010263 0.9827588  0.

In [27]:
total_record=[]
total_record=model.train(5000,total_record)

None


TypeError: 'NoneType' object is not iterable

In [ ]:
all_variables=[model.weights,model.biases]
print(all_variables)


[[<tf.Variable 'Variable:0' shape=(20, 1) dtype=float64, numpy=
array([[ 0.33419472],
       [ 0.27755291],
       [-0.09805934],
       [ 0.40352766],
       [-0.00509879],
       [ 0.10950405],
       [-0.05277721],
       [ 0.06969386],
       [ 0.32769571],
       [ 0.04114216],
       [-0.03858022],
       [ 0.29587913],
       [ 0.21606287],
       [-0.06228451],
       [-0.20560043],
       [-0.01487679],
       [-0.365012  ],
       [-0.61200223],
       [ 0.16868443],
       [ 0.21724071]])>, <tf.Variable 'Variable:0' shape=(20, 20) dtype=float64, numpy=
array([[-1.36813727e-01,  2.09748106e-01, -1.63044257e-02,
        -2.68833695e-01,  8.12093257e-03,  1.95084805e-01,
        -7.99277993e-02,  1.17006413e-01,  4.51607797e-02,
         1.10786714e-01,  3.14859786e-01, -2.28553717e-01,
        -3.73195404e-02, -2.67942230e-02,  1.24504948e-01,
         2.25323211e-01, -2.05892488e-01,  6.44971595e-02,
         1.70326251e-01,  4.34812760e-03],
       [-3.89187652e-01,  4.99648